## Below is the code to re-create Figure 5

### Topic: Making clusters with different IMFs

In [2]:
# Import necessary packages. 
from popstar import synthetic, evolution, atmospheres, reddening, ifmr
from popstar.imf import imf, multiplicity
import numpy as np
import pylab as py
import pdb
import os
import pylab as py

/Users/mwhosek/anaconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/locations.py:329: UserWarning: Extinction files not found in /Users/mwhosek/models/models_2019_07/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


In [3]:
# Build the isochrone for the cluster. We'll adopt an age of 100 Myr, 
# and a distance of 10 pc  with no extinction (so, we'll get absolute mags).
# We'll use the MIST evolution models here.
logAge = 8.0 # Age in log(years)
AKs = 0 # Ks filter extinction in mags
dist = 10 # distance in parsecs
metallicity = 0 # metallicity in [M/H]
atm_func = atmospheres.get_merged_atmosphere
evo = evolution.MISTv1()
filt_list = ['nirc2,Kp']

iso_dir = 'iso_mist_r1/'

if not os.path.exists(iso_dir):
    os.mkdir(iso_dir)

iso_imf = synthetic.IsochronePhot(logAge, AKs, dist, metallicity=metallicity,
                                 evo_model=evo, atm_func=atm_func,
                                  filters=filt_list,
                                 iso_dir=iso_dir, mass_sampling=1)

Isochrone generation took 100.583023 s.
Making photometry for isochrone: log(t) = 8.00  AKs = 0.00  dist = 10
     Starting at:  2019-11-06 14:53:16.614233   Usually takes ~5 minutes
Starting filter: nirc2,Kp   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.112 Msun  T =  3010 K  m_nirc2_Kp = 8.33
M =   1.612 Msun  T =  7715 K  m_nirc2_Kp = 2.10
M =   4.773 Msun  T = 13582 K  m_nirc2_Kp = -1.27
M =   5.013 Msun  T = 10084 K  m_nirc2_Kp = -2.77
M =   5.020 Msun  T =  4310 K  m_nirc2_Kp = -5.19
M =   5.271 Msun  T =  4854 K  m_nirc2_Kp = -5.37
M =   5.321 Msun  T =  3454 K  m_nirc2_Kp = -8.04
M =   5.324 Msun  T = 26655 K  m_nirc2_Kp = -3.09
M =   5.324 Msun  T = 192342 K  m_nirc2_Kp = 3.29
      Time taken: 19.25 seconds


In [4]:
# Define two different IMFs between 0.1 - 150 M_sun: one ``normal'' from 
# Kroupa+01 and the other top-heavy from Hosek+19. We will include multiplicity
# here.
#
# (technical note: Hosek+19 analysis only went down to ~2 M_sun, so here we are 
# assuming that the top-heavy IMF turns over at 0.5 M_sun,
# like Kroupa+01, and then is consistent with Kroupa+01 below 0.5 M_sun. This is
# not important for this example though)
clust_multiplicity = multiplicity.MultiplicityUnresolved()

massLimits = np.array([0.1, 0.5, 120])
powers_kroupa = np.array([-1.3, -2.3])
powers_theavy = np.array([-1.3, -1.7]) # top heavy
trunc_kroupa = imf.IMF_broken_powerlaw(massLimits, powers_kroupa, 
                                       multiplicity=clust_multiplicity)
trunc_theavy = imf.IMF_broken_powerlaw(massLimits, powers_theavy,
                                      multiplicity=clust_multiplicity)

In [5]:
# Make the clusters, using our IFMR. We'll adopt a mass of 10^6 M_sun
my_ifmr = ifmr.IFMR()
clust_mtot = 10**6

clust_kroupa = synthetic.ResolvedCluster(iso_imf, trunc_kroupa, clust_mtot, ifmr=my_ifmr)
clust_kroupa = clust_kroupa.star_systems

clust_theavy = synthetic.ResolvedCluster(iso_imf, trunc_theavy, clust_mtot, ifmr=my_ifmr)
clust_theavy = clust_theavy.star_systems

/Users/mwhosek/anaconda3/envs/astroconda/lib/python3.5/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, op)(other)
/Users/mwhosek/anaconda3/envs/astroconda/lib/python3.5/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, op)(other)
/Users/mwhosek/anaconda3/envs/astroconda/lib/python3.5/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in greater_equal
  return getattr(self.data, op)(other)


Found 75316 stars out of mass range
Found 20427 companions out of stellar mass range
Found 13990 stars out of mass range
Found 19910 companions out of stellar mass range


In [6]:
# Make 2 panel plot: left panel showing Kp luminosity functions, the 
# right panel showing compact objects

# Extra step to eliminate np.nans from the luminosity function; these are
# systems that only contain compact objects
good_k = np.where( np.isnan(clust_kroupa['m_nirc2_Kp']) == False)
good_th = np.where( np.isnan(clust_theavy['m_nirc2_Kp']) == False)

py.figure(figsize=(20,10))
py.subplots_adjust(left=0.08)
py.subplot(121)
hist, bins, patch = py.hist(clust_kroupa['m_nirc2_Kp'][good_k], bins=30, histtype='step',
                           linewidth=2, color='black', label='Kroupa+01')
py.hist(clust_theavy['m_nirc2_Kp'][good_th], bins=bins, histtype='step',
                           linewidth=2, color='red', label='Top-Heavy')
py.xlabel('NIRC2 Kp (absolute mag)', fontsize=24)
py.ylabel('N$_{stars}$', fontsize=24)
py.tick_params(axis='both', labelsize=20)
py.legend(loc=2, fontsize=20)
py.gca().set_yscale('log')


# For compact objects, let's look at the black hole mass function.
# Note that we need to pull mass_current to get mass of black hole
idx_k_bh = np.where(clust_kroupa['phase'] == 103)
idx_th_bh = np.where(clust_theavy['phase'] == 103)

py.subplot(122)
hist, bins, patch = py.hist(clust_kroupa['mass_current'][idx_k_bh], bins=15, histtype='step',
                           linewidth=2, color='black', label='Kroupa+01')
py.hist(clust_theavy['mass_current'][idx_th_bh], bins=bins, histtype='step',
                           linewidth=2, color='red', label='Top-Heavy')
py.xlabel('Mass (M$_{\odot}$)', fontsize=24)
py.ylabel('N$_{stars}$', fontsize=24)
py.tick_params(axis='both', labelsize=20)
py.legend(loc=1, fontsize=20)
py.gca().set_yscale('log')
py.savefig('imf_comp.pdf', format='pdf')